## Multi Agent Collaborative Content Generation (DHS-2025)

### Ojectives

* To showcase practical example of using Agents to solve problems.

* Provide a framwork which can be extended to other similar taks.

* Use of Local LLMs in Agents with Groq and Ollama

* Why and how of Agents.



<img src="assets/Lets Hack.png" alt="Generator Agent" style="width: 400px;"/>


**Install these libraries**

In [1]:
# !pip install python-dotenv
# !pip install pyyaml
# !pip install openai duckduckgo-search newspaper4k lxml_html_clean agno
# !pip install exa_py
# !pip install -U googlesearch-python pycountry

In [13]:
!python env_set_donot_share.py

In [3]:
## Import Helper Libraries

from typing import Dict, Iterator
from dotenv import load_dotenv
from datetime import datetime
from pathlib import Path
from textwrap import dedent
import yaml


## Import Agno Packages
from agno.utils.log import logger
from agno.utils.pprint import pprint_run_response, apprint_run_response
from agno.workflow import Workflow
from agno.team.team import Team
from agno.tools.googlesearch import GoogleSearchTools
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.agent import Agent, RunResponse
from agno.tools.exa import ExaTools
from agno.models.ollama import Ollama, OllamaTools
from agno.models.groq import Groq


## Import Local Utils
from utils import GoogleSearchSnippet


# Load API keys and environment variables
#load_dotenv()

**Load LLM as brain to Agents**

In [4]:
## Tip : Use LLMs which supports tool calls, if planning to use tools in Agents.

#cogito_model = Ollama(id="cogito:14b-v1-preview-qwen-q4_K_M")

#qwen4b_model = Ollama(id="qwen3:4b-q4_K_M")
#qwen1b_model = Ollama(id="qwen3:1.7b-q4_K_M")

#qwen1b_model = Groq(id="qwen-qwq-32b")

deepseek_model = Groq(id="deepseek-r1-distill-llama-70b")


**Lets start creating Agents**

Building agentic solution requires to break down the complex workflow into logical groups. 

In real world specialized Micro agents work better.[[Micro-Agents](https://github.com/humanlayer/12-factor-agents/blob/main/content/brief-history-of-software.md)]

Language models managing well-scoped sets of tasks makes it easy to incorporate

### Generator Agent



<img src="assets/Generator.jpg" alt="Generator Agent" style="width: 300px;"/>


Create a Generator Agent

In [6]:
snippet_generator = Agent(
        name="SEO Expert",
        role="You are SEO Expert specialist in Ecommerce. Write SEO-optimized snippets for query.",
        description="Write contextually relevant snippet by using details from Google Search.",
        model=deepseek_model,
        tools=[GoogleSearchTools()],
        add_name_to_instructions=True,
        debug_mode=False,
        instructions=dedent("""
            You will be given a keyword for which snippets needs to be generated.
            Include the keyword in the title and description both.
            Keep the snippet aligned to other ecommerce snippets on Search Page.
            Keep you response precise.
        """),
    )

### GuardRails Agent

**Why** -> Each generated snippet need to be validated to meet the requirements

1. Textual Relevance.
2. Grounding
3. Best Practices of SEO. (Title length, Description length, Readable, Precise)

<img src="assets/GuardRails.jpg" alt="Validator Agent" style="width: 300px;"/>

In [7]:
snippet_guardrails = Agent(
        name="SEO and Product Expert with E-commerce experience.",
        role="Validates SEO-optimized snippets for query.",
        model=deepseek_model,
        tools=[],
        add_name_to_instructions=True,
        instructions=dedent("""
            Your task is to validate the SEO-optimized snippets generated by the Snippet Generator.
            You will be given a keyword for which snippets needs to be validated.
            Your follow these guidelines:
            1. Check for keyword relevance and density.
            2. check if keyword in present in title and meta-description.
            3. Validate the snippet's structure and readability.
            4. Ensure the snippets adhere to SEO best practices.
            5. Do not include any offers or promotions and urls in the snippet.
            6. Keep your response precise.
        """),
    )

### Improver Agent

**Why** -> Improver agent co-ordinates on feedback provided by GuardRails agent. It re-writes the snippet based on GuardRails Agent feedback.

1. Ensures consistency and relevance.
2. Make changes based on feedback.


<img src="assets/ImproverAgent.jpg" alt="Improver Agent" style="width: 300px;"/>

In [8]:
snippet_improver = Agent(
        name="Snippet Improver",
        role="SEO and Product Expert with E-commerce experience. Correct the snippet for problems identified by the validator.",
        model=deepseek_model,
        tools=[],
        add_name_to_instructions=True,
        instructions=dedent("""
            Your task is to Correct the snippets by considering the problems identified by the validator.
            You will be given a snippet for which problems needs to be corrected.
            Your follow these guidelines:
            1. Introduce the change only for the problems identified by the validator.
            2. Keep the snippets as close to the original as possible.
            3. Keep you response precise.
        """)
    )

### Structured Output Agent

**Why** -> Make sure that output schema is consistent and valid to consume for down-stream task.

1. Fixed outout for analysis and use.
2. Help reduce in hallucination, slight compermise in output quality.


<img src="assets/OutputAgent.jpg" alt="Output Agent" style="width: 300px;"/>

In [9]:
snippet_writer = Agent(
        description="Write the snippet in the json format produced by snippet improver.",
        instructions= dedent( """ Strict output format of the snippet in the json format."""),
        response_model=GoogleSearchSnippet,
        model=deepseek_model,
        add_name_to_instructions=True,
        role="Writes the snippet in the json format.",
    )

### Team of Agent

**Why** -> A coordinator agent manages the team, breaking down tasks and assigning them to appropriate specialist agents, who then collaborate to produce a final result. 

1. Delegation.
2. synthesizes Team Agent outputs and create responses.
3. Parallel / Sequential execution.


<img src="assets/Team.jpg" alt="Output Agent" style="width: 500px;"/>

In [10]:
class TeamWorkflow(Workflow):
    description: str = (
        "Write SEO-optimized snippets for query. Which covers the query and user intents."
    )

    snippet_generator = snippet_generator
    snippet_guardrails = snippet_guardrails
    snippet_improver = snippet_improver
    snippet_writer = snippet_writer
    
    agent_team = Team(
        name="SEO Expert Team",
        mode="collaborate",
        model=deepseek_model,
        members=[
            snippet_generator,
            snippet_guardrails,
            snippet_improver,
            snippet_writer    
        ],
        instructions=[
            "You are a coordinator.",
            "Your primary role is to facilitate the snippet generation process.",
            "Once team members have provided their results you should stop the discussion.",
            "Do not continue the discussion after receiving the valid snippet.",
            "Ensure each member provides relevant output according to the role and instructions.",
            "Keep you response precise."
        ],
        success_criteria="The team has reached a consensus.",
        enable_agentic_context=True,
        show_tool_calls=True,
        markdown=True,
        debug_mode=False,
        show_members_responses=True,
    )
    
    def run(self, prompt, keyword=None) -> Iterator[RunResponse]:
        logger.info(f"Writing snippets for {self.run_id}")
        discussion: RunResponse = self.agent_team.run(
            #"Write snippet for the keyword: " + keyword
            prompt + keyword
        )
            
        if discussion is None:
            yield RunResponse(
                run_id=self.run_id, content="Sorry, could not generate the snippet."
            )
            return

        logger.info(f"Writing Snippet for keyword: {keyword}")
        yield from self.snippet_writer.run(discussion.content, stream=True)

### Agents in Action

In [22]:
report: Iterator[RunResponse] = TeamWorkflow(debug_mode=False)\
    .run(prompt="Generate snippet for keyword: ", keyword="purge mask halloween")

output = list(report)

INFO Writing snippets for b26dc1c4-f540-4ba2-9ca4-64ad5b382092

INFO Writing Snippet for keyword: purge mask halloween

In [23]:
print(output[0].content.model_dump_json(indent=4))

{
    "title": "Unleash Terror with Our Purge Mask This Halloween!",
    "metadescription": "Discover the ultimate Halloween accessory with our authentic Purge mask, designed for comfort and durability. Perfect for parties, costumes, or pranks—shop now and make this Halloween unforgettable!"
}


In [24]:
report: Iterator[RunResponse] = TeamWorkflow(debug_mode=False)\
    .run(prompt="Generate snippet for keyword: ", keyword="decorative backsplash")
output = list(report)

print(output[0].content.model_dump_json(indent=4))

INFO Writing snippets for 4a7b46dc-938b-4e22-bd38-bc7e1e3da511

INFO Writing Snippet for keyword: decorative backsplash

{
    "title": "Elevate Your Kitchen with Stunning Decorative Backsplashes",
    "metadescription": "Discover the perfect decorative backsplash to transform your kitchen with stylish, durable options from modern glass to natural stone, combining functionality and beauty."
}


In [26]:
report: Iterator[RunResponse] = TeamWorkflow(debug_mode=False)\
    .run(prompt="Generate snippet for keyword: ", keyword="Disney princess toys for toddlers")
output = list(report)

print(output[0].content.model_dump_json(indent=4))

INFO Writing snippets for fee3b6c8-c400-47fe-9ce8-fb9a666cc311

INFO Writing Snippet for keyword: Disney princess toys for toddlers

{
    "title": "Disney Princess Toys for Toddlers | Safe & Educational",
    "metadescription": "Discover adorable Disney Princess toys designed for toddlers. Made with safe, non-toxic materials, these toys promote imaginative play and developmental skills. Perfect for little hands and big imaginations!"
}
